# Road Following - Live demo

In this notebook, we will use model we trained to move jetBot smoothly on track. 

In [1]:
import torchvision
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot

Inicializando modelo:

In [2]:
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_steering_model_xy.pth'))
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()
def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Inicializando câmera:

In [3]:
camera = Camera()

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

#display(image_widget)

Inicializando robô:

In [4]:
robot = Robot()

Configurando widgets:

In [5]:
'''
Implementação do botão e sliders
'''
autopilot_checkbox = ipywidgets.Checkbox(value=False,description='Autopilot')
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(autopilot_checkbox,speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

Checkbox(value=False, description='Autopilot')

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

Rotina de "road following":

In [6]:
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    STEERING = 0.025 
    SPEED = 0.33
    KD = 0.00
    image = change['new']
    target = model(preprocess(image))
    xy = target.detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y

    if autopilot_checkbox.value:
        speed_slider.value = SPEED
    else:
        speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    if autopilot_checkbox.value:
        pid = angle * (STEERING + (abs(np.sin(angle) * 0.03) / 2)) + (angle - angle_last) * KD
    else:
        pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
        
execute({'new': camera.value})

Iniciando rotina no robô:

In [7]:
camera.observe(execute, names='value')

Saída da câmera:

In [8]:
widget_width = camera.width
widget_height = camera.height

target_widget = ipywidgets.Image(format='jpeg', width=widget_width, height=widget_height)

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * widget_width / 2 + widget_width / 2)
    delta_x = abs(x - (widget_width / 2))
    y = int(y * widget_height / 2 + widget_height / 2)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (int(widget_width / 2), widget_height), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (int(widget_width / 2), widget_height), (255,0,0), 3)
#     image = cv2.putText(image, str(COUNT), (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    image = cv2.putText(image, str(0.025 + (abs(np.sin(angle) * 0.03) / 2)), (50, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)
display(target_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Parando robô:

In [37]:
import time
camera.unobserve(execute, names='value')
time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
robot.stop()


Desligando câmera:

In [ ]:
camera.stop()